# The method is as follows:
    
1. Guess the flows in each pipe, making sure that the total in flow is equal to the total out flow at each junction. (The guess doesn't have to be good, but a good guess will reduce the time it takes to find the solution.)   
2. Determine each closed loop in the system.    
3. For each loop, determine the clockwise head losses and counter-clockwise head losses. Head loss in each pipe is calculated using  $h_f = r Q^{n}$. Clockwise head losses are from flows in the clockwise direction and likewise for counter-clockwise.
4. Determine the total head loss in the loop, $\Sigma r Q^{n}$, by subtracting the counter-clockwise head loss from the clockwise head loss.
5. For each loop, find $\Sigma n r Q^{n - 1}$ without reference to direction (all values should be positive).
6. The change in flow is equal to $\frac{\Sigma r Q^{n}}{\Sigma n r Q^{n -1}}$ .
7. If the change in flow is positive, apply it to all pipes of the loop in the counter-clockwise direction. If the change in flow is negative, apply it to all pipes of the loop in the clockwise direction.
8. Continue from step 3 until the change in flow is within a satisfactory range.

In [1]:
from numpy import array as arr

In [2]:
#n = 2
P_servici =  10 + 1 #metri
Q = {'a': 10 + 1, #litru/secunda
     'b': 12 + 1,
     'c': 8.5 + 1,
     'd': 11 + 1,
     'e': 7.5 + 1,
     'q_dist': 0.01, #litru/ (secunda x metru)
     }
Q_in = 58 #l/s
D = {'3': 50, #mm
     '6':75,
     '15':125,
     '40':150,
     '75':200,
     '120':250,
     '180': 300, #valorile reprezinta pragul de schimbare al diametrului in fct de debit
    
    }

In [3]:
def reverse(string):
    new_string = ''
    for char in string:
        new_string = char + new_string
        print(new_string)
    return new_string

def hardy_cross_M(L, D): 
    lamda = 0.021 / pow(D, 0.3)
    return 0.0826 * lamda * pow(L, 2) / pow(D, 5)

def hardy_cross_numerator(M, Q, n = 2):
    return M*Q*pow(abs(Q), n-1)

def hardy_cross_denominator(M, Q, n = 2):
    return n*M*pow(abs(Q), n-1)

def hardy_cross_flow_correction(numerator, denominator):
    return -1* numerator/denominator

In [4]:
#solutii initiale            # nume, debit, diametru, lungime

loop_0 = { '1-2':  [ 28 * pow(10,-3), 150 * pow(10,-3) , 200], # trandf dim l ->m^3, mm ->m
           '2-6':  [  1 * pow(10,-3),  50 * pow(10,-3) , 150],
           '6-5':  [ -4 * pow(10,-3),  75 * pow(10,-3) , 350],
           '5-1':  [-28 * pow(10,-3), 150 * pow(10,-3) , 200]
                                                                }
loop_1 = { '2-3':  [ 15 * pow(10,-3), 125 * pow(10,-3) , 400],
           '3-4':  [  3 * pow(10,-3),  75 * pow(10,-3) , 200],
           '4-6':  [  3 * pow(10,-3),   5 * pow(10,-3) , 250],
           '6-2':  [ -1 * pow(10,-3),  50 * pow(10,-3) , 150]
                                                                }
loop_2 = { '6-4':  [  4 * pow(10,-3),  75 * pow(10,-3) , 350],
           '4-5':  [ -3 * pow(10,-3),   5 * pow(10,-3) , 250],
           '5-6':  [-11 * pow(10,-3), 125 * pow(10,-3) , 100]
                                                                } 

loops = [loop_0, loop_1, loop_2]

module_rezistenta = {'loop 0': [],
                     'loop 1': [],
                     'loop 2': []}

def M_appender(loop, M_key):
    for loop_data in loop.values():
            D = loop_data[1]
            L = loop_data[2]
            M = hardy_cross_M(L = L, D = D)
            module_rezistenta[M_key].append(M)

for i, key in enumerate(module_rezistenta.keys()):
    M_appender(loops[i], key)
    
print (module_rezistenta)

{'loop 0': [1614268.0842028141, 306789744.11046475, 194764919.083715, 1614268.0842028141], 'loop 1': [16970563.213011153, 63596708.272233464, 170035004178487.8, 306789744.11046475], 'loop 2': [194764919.083715, 170035004178487.8, 1060660.200813197]}


$M = 0.0826 \frac{{\lambda L^2}}{{D^5}}$

$\lambda = \frac {0.021}{D^{0.3}}$


In [5]:
def hardy_cross_loop_correction(M, Q, pipes_name = None): #peticita but works
    numerator_sum = 0
    denominator_sum = 0
    Q = list(Q.values())
    for i in range(len(Q)):
        numerator_sum += hardy_cross_numerator(M[i], Q[i][0])
        denominator_sum += hardy_cross_denominator(M[i], Q[i][0])
    
    delta_Q = hardy_cross_flow_correction(numerator_sum, denominator_sum)
    
    return delta_Q

In [6]:
#def loop_equilibrator()
#    
#    while (True):
#        if (delta_Q1 - delta_Q2) < :
#            break
            

while (True):
    
    delta_Q0 = hardy_cross_loop_correction( M = module_rezistenta['loop 0'], Q = loops[0], pipes_name = loops[0].keys() )
    delta_Q1 = hardy_cross_loop_correction( M = module_rezistenta['loop 1'], Q = loops[1], pipes_name = loops[1].keys() )
    delta_Q2 = hardy_cross_loop_correction( M = module_rezistenta['loop 2'], Q = loops[2], pipes_name = loops[2].keys() )
    
    delta_Q = [delta_Q0, delta_Q1, delta_Q2]
    
    
    for k, loop in enumerate(loops):
        loop_params = list(loop.values())
        for pipe_data in loop_params:
            debit = pipe_data[0]
            debit += delta_Q[k]
            
        print(new_data)    
            
    
    if ((dQ01-dQ1) <= dQ03/100) and ((dQ02-dQ2) <= dQ03/100) and ((dQ03-dQ3) <= dQ03/100):
        break

